In [1]:
import os
import imp
import numpy as np
import theano
import lasagne
import loading
from training import *
from network import *
from architectures import *

L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor

headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/temp')
datafile = os.path.join(headdir, 'Data/0_hvh/Clean/_summaries/model_input_with_groups.csv')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nns')
data = loading.default_loader(datafile)

## Basic Prototyping

In [2]:
# CHANGES SINCE PREVIOUS BEST:
# - implementation. Now with better abstraction! 
#     Should have no effect on performance, but dev much easier.
# - training algo. Now using Adam instead of SBGD with Nesterov momentum 
#     mostly just faster convergence...)
# - changed activation on conv2d layer to be a pRelu with all units fitted (previously vanilla Relu)
# - changed activation on output to be leaky Relu with alpha=.3
# - changed initialization on output and conv2d layers to be He with proper func instead of Glorot
# - added a "feature pool" layer after conv2d. 
#     This was most impactful; improved performance by ~.05 and reduced overfitting

In [3]:
def default_convnet(input_var=None):
    input_shape = (None, 2, 4, 9)
    FixLayer = make_FixLayer(input_var)
    
    input_layer = L.InputLayer(shape=input_shape, input_var=input_var)
    network = L.Conv2DLayer(
        input_layer, num_filters=32, filter_size=(4,4), pad='full',
        nonlinearity=nl.identity
    )
    
    network = L.ParametricRectifierLayer(network, shared_axes='auto') # default: auto
    network = L.FeaturePoolLayer(network, pool_function=T.sum, pool_size=2) # default: T.sum, 2
    network = L.DropoutLayer(network, p=.75) # default: .75
    network = L.DenseLayer(
        network, num_units=36, 
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu')
    )
    network = FixLayer(network)
    network = L.NonlinearityLayer(network, nonlinearity=nl.softmax)
    network = FixLayer(network)
    network = ReNormLayer(network)
    
    return network

net = Network(default_convnet)

In [ ]:
trainer = DefaultTrainer(stopthresh=100) # default: 125
net_list = trainer.train_all(architecture=default_convnet, data=data, seed=985227)

Epoch 0 took 1.390s
	training loss:			2.9994
	validation loss:		2.8461
	validation accuracy:		15.53%
	total time elapsed:		1.413s


/Users/gianni/anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


Epoch 50 took 1.333s
	training loss:			2.1766
	validation loss:		2.1221
	validation accuracy:		34.86%
	total time elapsed:		71.326s
Epoch 100 took 1.335s
	training loss:			2.0851
	validation loss:		2.0693
	validation accuracy:		36.33%
	total time elapsed:		139.673s
Epoch 150 took 1.344s
	training loss:			2.0626
	validation loss:		2.0521
	validation accuracy:		37.11%
	total time elapsed:		207.949s
Epoch 200 took 1.430s
	training loss:			2.0516
	validation loss:		2.0408
	validation accuracy:		37.60%
	total time elapsed:		279.748s
Epoch 250 took 1.365s
	training loss:			2.0411
	validation loss:		2.0461
	validation accuracy:		37.21%
	total time elapsed:		350.684s
Epoch 300 took 1.377s
	training loss:			2.0343
	validation loss:		2.0318
	validation accuracy:		37.60%
	total time elapsed:		420.298s
Epoch 350 took 1.389s
	training loss:			2.0131
	validation loss:		2.0311
	validation accuracy:		37.70%
	total time elapsed:		490.723s
Epoch 400 took 1.444s
	training loss:			1.9953
	validation loss:

## Subject prototyping

TODO:

try saving parameters from above and doing a reload/partial freeze

In [28]:
def subject_convnet(input_var=None):
    input_shape = (None, 2, 4, 9)
    FixLayer = make_FixLayer(input_var)
    
    network = L.InputLayer(shape=input_shape, input_var=input_var)
    network = L.Conv2DLayer(
        network, num_filters=32, filter_size=(4,4), pad='full',
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu') #GlorotUniform()
    )
    
#     network = L.ParametricRectifierLayer(network, shared_axes='auto')
    network = L.DropoutLayer(network, p=.75)

    network = L.FeaturePoolLayer(network, pool_function=T.sum, pool_size=8)

    network = L.DropoutLayer(network, p=.75)
    network = L.DenseLayer(
        network, num_units=36, 
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu')
    )
    network = L.NonlinearityLayer(network, nonlinearity=nl.softmax)
    network = FixLayer(network)
    
    return network


test_sub = 10
subject_Xs = data[2]
subject_ys = data[3]
subject_Ss = data[4]

S0 = [np.where(s==test_sub)[0] for s in subject_Ss]
print(len(S0[0]))
X0 = [subject_Xs[i][s] for i, s in enumerate(S0)]
y0 = [subject_ys[i][s] for i, s in enumerate(S0)]

sub0_data = (data[0], data[1], X0, y0, S0)
sub0_trainer = DefaultTrainer(batchsize=16, stopthresh=20, update_args={'learning_rate':.000001})
sub0_net_list = sub0_trainer.train_all(architecture=subject_convnet, data=sub0_data, seed=985227)

17
Epoch 0 took 0.023s
	training loss:			6.1382
	validation loss:		3.5240
	validation accuracy:		6.25%
	total time elapsed:		0.024s
Epoch 50 took 0.022s
	training loss:			2.5513
	validation loss:		2.5485
	validation accuracy:		18.75%
	total time elapsed:		1.127s
Abandon ship!
TEST PERFORMANCE
	Stopped in epoch:		74
	Test loss:			2.8323
	Test accuracy:			6.25
Epoch 0 took 0.022s
	training loss:			6.6821
	validation loss:		3.4139
	validation accuracy:		6.25%
	total time elapsed:		0.022s
Epoch 50 took 0.022s
	training loss:			2.5866
	validation loss:		2.7526
	validation accuracy:		6.25%
	total time elapsed:		1.144s
Abandon ship!
TEST PERFORMANCE
	Stopped in epoch:		90
	Test loss:			2.9185
	Test accuracy:			0.00
Epoch 0 took 0.023s
	training loss:			6.2076
	validation loss:		3.2042
	validation accuracy:		12.50%
	total time elapsed:		0.023s
Abandon ship!
TEST PERFORMANCE
	Stopped in epoch:		45
	Test loss:			2.6970
	Test accuracy:			18.75
Epoch 0 took 0.024s
	training loss:			6.1728
	validat

## Autoencoder prototyping

In [ ]:
def autoencoder_prototype(input_var=None):
    input_shape = (None, 2, 4, 9)
    inpuL.InputLayer = L.InputLayer(shape=input_shape, input_var=input_var)
    network = L.COnv2DLayer(
        input_layer, num_filters=32, filter_size=(4, 4), pad='full',
        nonlinearity=nl.identity
    )
    L.ParametricRectifierLayerrLayeshared_axes=Rectifier(network, shL.FeaturePoolLayerl')
    network = L.FeaturePoolLayer(network, pool_functioL.DropoutLayerm, pool_size=2)
    network = L.DropoutLayer(network, p=.75)
    network = L.DenseLayer(
        network, num_units=72,
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu')
    )
    network = L.ReshapeLayer(
        network, shape=(-1, 2, 4, 9)
    )
    return network

In [ ]:
class Autoencoder(Network):
    """
    Overwrites loss functions for training an autoencoder instead
    of softmax output classifier
    """
    def objectives(self):
        """Must define: loss, test_loss, test_acc"""
        self.loss = lasagne.objectives.squared_error(self.prediction, self.target_var)
        self.loss = self.loss.mean()
        self.test_loss = lasagne.objectives.squared_error(self.test_prediction, self.target_var)
        self.test_loss = self.test_loss.mean()
        self.test_acc = T.mean(T.abs(self.test_prediction - self.target_var))
        return None

# class AutoTrainer(DefaultTrainer):
#     """
#     Overwrites training functions for training an autoencoder instead
#     of softmax output classifier
#     """
#     pass

# Can actually pass data to existing trainer by replacing ys with a copy of Xs!

class LadderNet(Network):
    """
    Loads conv layer params from autoencoder save and sets them on current classifier
    Needs overwrite of self.build() that loads conv2d params, then sets them untrainable.
    """
    def __init__(self, architecture, paramsfile):
        self.paramsfile = paramsfile
        super(LadderNet, self).__init__(architecture)
    
    def build(self):
        super(LadderNet, self).build()
        
        # Finish this soon!
        return None
    
    def load_conv_params(self):
        with np.load(paramsfile) as loaded:
            L.

In [ ]:
from scipy.stats import bayes_mvs as bmvs
bmvs([n.test_err for n in net_list], alpha=.95)

In [ ]:


D, groups, Xs, ys, Ss = loading.default_loader(datafile)

splitsize = len(Xs)
r = np.arange(splitsize)
r = np.tile(r, [splitsize, 1])
r = r + r.T
split_indices = r % splitsize

split = 0
train_i = split_indices[split, :3]
val_i = split_indices[split, 3:4]
test_i = split_indices[split, 4:]

X, y, S = [np.concatenate(np.array(Zs)[train_i]) for Zs in [Xs, ys, Ss]]
Xv, yv, Sv = [np.concatenate(np.array(Zs)[val_i]) for Zs in [Xs, ys, Ss]]
Xt, yt, St = [np.concatenate(np.array(Zs)[test_i]) for Zs in [Xs, ys, Ss]]
X, y = loading.augment((X, y))
S = np.concatenate([S, S, S, S])

In [ ]:
np.random.seed(985227)

net = Network(default_convnet)
trainer = Trainer()

trainer.train(net, training_data=(X, y), validation_data=(Xv, yv))
err, acc, bats = trainer.test(net, (Xt, yt))